In [1]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph

from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from pydantic import BaseModel, Field
# from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer

from langchain_neo4j import Neo4jVector
# from langchain_openai import OpenAIEmbeddings
from langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars

c:\Users\Subrata Samanta\miniconda3\envs\genai\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [25]:
from langchain_groq import ChatGroq

load_dotenv()
chat = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile")





print(chat.invoke("Write a poem about time travel."))

content="Through the veil of time, a doorway wide,\nA portal to the past, where secrets reside.\nA journey through the ages, to eras gone by,\nTo witness history's birth, and watch the moments fly.\n\nWith chronometer in hand, and a heart full of cheer,\nThe time traveler embarks, on a voyage so dear.\nThrough swirling vortex, and a tunnel of light,\nThey soar through the decades, into the dark of night.\n\nIn ancient lands, they walk among the brave,\nAnd witness empires rise, and civilizations wave.\nThey see the pyramids, in all their golden glow,\nAnd hear the whispers of pharaohs, in the desert's flow.\n\nIn medieval castles, they dine with kings and queens,\nAnd dance to the minstrels' tunes, in a bygone scene.\nThey visit the Renaissance, where art and science thrive,\nAnd meet the masters, who shaped the world alive.\n\nBut time travel's a curse, as well as a blessing true,\nFor every step forward, is a step away from you.\nThe traveler's heart, is torn between two eras apart,\

In [3]:
from euriai import EuriaiLangChainLLM

chat = EuriaiLangChainLLM(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4.1-nano",
    temperature=0.7,
    max_tokens=30000
)
print(chat.invoke("Write a poem about time travel."))

Through veils of years, where moments intertwine,  
A whisper beckons from the edge of time.  
Footsteps on the sands of days gone by,  
A leap beyond the present, soaring high.

In ancient halls where echoes softly dwell,  
Or future sights where dreams and fears swell,  
Time's ribbon bends, unbound, unseen,  
A voyage through what has been and what might have been.

With every tick, a portal opens wide,  
A chance to glimpse the past or future’s tide.  
Yet, in this dance of seconds spun anew,  
Lies wisdom, wonder, and a mystery true.

So drift beyond the now, embrace the flow,  
In time’s vast ocean, let your spirit go.  
For in each moment, fleeting, bold, and free—  
Lies all of history's eternity.


In [3]:

AURA_INSTANCENAME = os.environ["AURA_INSTANCENAME"]
NEO4J_URI = os.environ["NEO4J_URI"]
NEO4J_USERNAME = os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD = os.environ["NEO4J_PASSWORD"]
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT")

In [4]:
kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
) #database=NEO4J_DATABASE,


In [9]:
# # read the wikipedia page for the Roman Empire
from langchain_community.document_loaders import PyPDFLoader

raw_documents = PyPDFLoader(r"Annual Report\NASDAQ_AAPL_2024.pdf").load()



In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# # # # Define chunking strategy
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
documents = text_splitter.split_documents(raw_documents)
print(len(documents))


# graph_documents = llm_transformer.convert_to_graph_documents(documents)

448


In [ ]:
# from concurrent.futures import ThreadPoolExecutor
# from tqdm import tqdm
# import pickle

# llm_transformer = LLMGraphTransformer(llm=chat)

# # Function to process a single document
# def process_document(doc):
#     return llm_transformer.convert_to_graph_documents([doc])


# graph_documents = []
# with ThreadPoolExecutor() as executor:
#     with tqdm(total=len(documents), desc="Processing Documents") as pbar:
#         for i in range(0, len(documents), 100):  # Batch size of 100
#             batch = documents[i:i+100]
#             futures = {executor.submit(process_document, doc): doc for doc in batch}
#             for future in futures:
#                 result = future.result()
#                 graph_documents.extend(result)
#                 pbar.update(1)

# # Save graph_documents to a pickle file
# with open("graph_documents.pkl", "wb") as f:
#     pickle.dump(graph_documents, f)


Processing Documents: 100%|██████████| 10/10 [03:37<00:00, 21.72s/it]


In [12]:
# Load graph_documents from the pickle file
import pickle
with open("graph_documents.pkl", "rb") as f:
    graph_documents = pickle.load(f)

print(f"Loaded {len(graph_documents)} graph documents.")

Loaded 448 graph documents.


In [ ]:
# # # store to neo4j
# res = kg.add_graph_documents(
#     graph_documents,
#     include_source=True,
#     baseEntityLabel=True,
# )


[#C77E]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.126.161.242', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))): OSError('No data')
Unable to retrieve routing information
Transaction failed and will be retried in 1.0212015055032122s (Unable to retrieve routing information)
[#C77F]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-1bba48b4-8448.production-orch-0695.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))): OSError('No data')
Transaction failed and will be retried in 1.9343195368755242s (Failed to read from defunct connection IPv4Address(('si-1bba48b4-8448.production-orch-0695.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))))
[#C854]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-1bba48b4-8448.production-orch-0695.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))): OSError('No data')
Transaction failed and will be ret

In [14]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [15]:
# create vector index
vector_index = Neo4jVector.from_existing_graph(
    hf,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)


In [45]:
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities or other graph entities that "
        "appear in the text",
    )


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following, there can be multiple entities "
            "input: {question}",
        ),
    ]
)


In [46]:
entity_chain = prompt | chat.with_structured_output(Entities)

In [47]:
# # Test it out:
res = entity_chain.invoke(
    {"question": "How has the company's capital expenditure (CapEx) evolved over the past five years, and what strategic initiatives have driven significant changes in CapEx allocations"}
).names
print(res)

['company']


In [33]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [56]:
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        print(f" Getting Entity: {entity}")
        # Using standard pattern matching instead of fulltext search
        response = kg.query(
            """
            MATCH (node)
            WHERE node.name =~ $query 
            OR node.id =~ $query
            WITH node
            MATCH (node)-[r]->(neighbor)
            RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
            UNION ALL
            MATCH (node)<-[r]-(neighbor)
            WHERE node.name =~ $query 
            OR node.id =~ $query
            RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
            LIMIT 50
            """,
            {"query": f"(?i).*{entity}.*"}  # Case-insensitive pattern matching
        )
        result += "\n".join([el["output"] for el in response])
    return result

In [57]:
print(structured_retriever("what is the Company"))


 Getting Entity: Company
Company - COMPLIANT_WITH -> Gaap
Company - STORES -> Confidential Information
Company - MARKET -> Products And Services
Company - FOCUS -> Innovation
Company - PRODUCES -> Products
Company - HOLDS -> Intellectual Property
Company - RELIES_ON -> Personnel
Company - OBTAIN -> Licenses
Company - HIGHER_NET_SALES -> First Quarter
Company - BELIEF -> Compensation
Company - INVESTS_IN -> Growth And Development
Company - OFFERS_PROGRAMS_VIA -> Apple University
Company - COMMITTED_TO -> Workplace Practices And Policies
Company - CREATES -> Confidential Information
Company - IDENTIFIES -> Workplace Risks
Company - MITIGATES -> Hazards
Company - SUPPORTS -> Employees
Company - AFFECTED_BY -> Effective Tax Rates
Company - HAS_DIFFICULTY_WITH -> Manufacture And Deliver Products
Company - INCREASES -> Costs
Company - NEGATIVELY_IMPACTS -> Consumer Spending And Demand
Company - REQUIRE -> Additional Agreements
Company - PARTICIPATES_IN -> Global Smartphone, Personal Computer

In [58]:
# Final retrieval step
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [
        el.page_content for el in vector_index.similarity_search(question)
    ]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    print(f"\nFinal Data::: ==>{final_data}")
    return final_data

In [59]:
# Define the RAG chain
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


In [60]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | chat
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x: x["question"]),
)


In [61]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | chat
    | StrOutputParser()
)

In [63]:
res_hist = chain.invoke(
    {
        "question": "How has the company's capital expenditure (CapEx) evolved over the past five years, and what strategic initiatives have driven significant changes in CapEx allocations?",
        "chat_history": [

        ],
    }
)

print(f"\n === {res_hist}\n\n")

Search query: How has the company's capital expenditure (CapEx) evolved over the past five years, and what strategic initiatives have driven significant changes in CapEx allocations?
 Getting Entity: company

Final Data::: ==>Structured data:
Company - COMPLIANT_WITH -> Gaap
Company - STORES -> Confidential Information
Company - MARKET -> Products And Services
Company - FOCUS -> Innovation
Company - PRODUCES -> Products
Company - HOLDS -> Intellectual Property
Company - RELIES_ON -> Personnel
Company - OBTAIN -> Licenses
Company - HIGHER_NET_SALES -> First Quarter
Company - BELIEF -> Compensation
Company - INVESTS_IN -> Growth And Development
Company - OFFERS_PROGRAMS_VIA -> Apple University
Company - COMMITTED_TO -> Workplace Practices And Policies
Company - CREATES -> Confidential Information
Company - IDENTIFIES -> Workplace Risks
Company - MITIGATES -> Hazards
Company - SUPPORTS -> Employees
Company - AFFECTED_BY -> Effective Tax Rates
Company - HAS_DIFFICULTY_WITH -> Manufacture A

In [64]:
res_hist = chain.invoke(
    {
        "question": "What are the key risks identified by the company related to emerging technologies, such as artificial intelligence, and how has the company's risk mitigation strategy evolved in response to these technologies??",
        "chat_history": [

        ],
    }
)

print(f"\n === {res_hist}\n\n")

Search query: What are the key risks identified by the company related to emerging technologies, such as artificial intelligence, and how has the company's risk mitigation strategy evolved in response to these technologies??
 Getting Entity: artificial intelligence
 Getting Entity: company

Final Data::: ==>Structured data:
Artificial Intelligence Features - INCREASE_RISK -> Safety
Artificial Intelligence Features - INCREASE_RISK -> Security
c0305fe230844c0991fb4e92ad6001be - MENTIONS -> Artificial Intelligence Features
4102a4b6f3f7eaab0fcfe2199dee1ddd - MENTIONS -> Machine Learning And Artificial Intelligence
5983508620cd0ededc17b082564a1f94 - MENTIONS -> Machine Learning And Artificial Intelligence
Laws And Regulations - INCLUDES -> Machine Learning And Artificial Intelligence
Company - EXPAND_USE_OF -> Machine Learning And Artificial IntelligenceCompany - COMPLIANT_WITH -> Gaap
Company - STORES -> Confidential Information
Company - MARKET -> Products And Services
Company - FOCUS -> 

In [65]:
res_hist = chain.invoke(
    {
        "question": "How does the company's approach to sustainability reporting align with industry best practices, and what measurable outcomes have been achieved in environmental, social, and governance (ESG) areas over the past three years?",
        "chat_history": [

        ],
    }
)

print(f"\n === {res_hist}\n\n")

Search query: How does the company's approach to sustainability reporting align with industry best practices, and what measurable outcomes have been achieved in environmental, social, and governance (ESG) areas over the past three years?
 Getting Entity: company

Final Data::: ==>Structured data:
Company - COMPLIANT_WITH -> Gaap
Company - STORES -> Confidential Information
Company - MARKET -> Products And Services
Company - FOCUS -> Innovation
Company - PRODUCES -> Products
Company - HOLDS -> Intellectual Property
Company - RELIES_ON -> Personnel
Company - OBTAIN -> Licenses
Company - HIGHER_NET_SALES -> First Quarter
Company - BELIEF -> Compensation
Company - INVESTS_IN -> Growth And Development
Company - OFFERS_PROGRAMS_VIA -> Apple University
Company - COMMITTED_TO -> Workplace Practices And Policies
Company - CREATES -> Confidential Information
Company - IDENTIFIES -> Workplace Risks
Company - MITIGATES -> Hazards
Company - SUPPORTS -> Employees
Company - AFFECTED_BY -> Effective T

In [66]:
res_hist = chain.invoke(
    {
        "question": "What are the trends in the company's revenue diversification across different geographical regions, and how have geopolitical factors influenced these trends over the past five years?",
        "chat_history": [

        ],
    }
)

print(f"\n === {res_hist}\n\n")

Search query: What are the trends in the company's revenue diversification across different geographical regions, and how have geopolitical factors influenced these trends over the past five years?
 Getting Entity: company

Final Data::: ==>Structured data:
Company - COMPLIANT_WITH -> Gaap
Company - STORES -> Confidential Information
Company - MARKET -> Products And Services
Company - FOCUS -> Innovation
Company - PRODUCES -> Products
Company - HOLDS -> Intellectual Property
Company - RELIES_ON -> Personnel
Company - OBTAIN -> Licenses
Company - HIGHER_NET_SALES -> First Quarter
Company - BELIEF -> Compensation
Company - INVESTS_IN -> Growth And Development
Company - OFFERS_PROGRAMS_VIA -> Apple University
Company - COMMITTED_TO -> Workplace Practices And Policies
Company - CREATES -> Confidential Information
Company - IDENTIFIES -> Workplace Risks
Company - MITIGATES -> Hazards
Company - SUPPORTS -> Employees
Company - AFFECTED_BY -> Effective Tax Rates
Company - HAS_DIFFICULTY_WITH -

In [67]:
res_hist = chain.invoke(
    {
        "question": "How has the company's debt profile changed over the last five years, and what implications does this have for its financial stability and investment capacity?",
        "chat_history": [

        ],
    }
)

print(f"\n === {res_hist}\n\n")

Search query: How has the company's debt profile changed over the last five years, and what implications does this have for its financial stability and investment capacity?
 Getting Entity: company

Final Data::: ==>Structured data:
Company - COMPLIANT_WITH -> Gaap
Company - STORES -> Confidential Information
Company - MARKET -> Products And Services
Company - FOCUS -> Innovation
Company - PRODUCES -> Products
Company - HOLDS -> Intellectual Property
Company - RELIES_ON -> Personnel
Company - OBTAIN -> Licenses
Company - HIGHER_NET_SALES -> First Quarter
Company - BELIEF -> Compensation
Company - INVESTS_IN -> Growth And Development
Company - OFFERS_PROGRAMS_VIA -> Apple University
Company - COMMITTED_TO -> Workplace Practices And Policies
Company - CREATES -> Confidential Information
Company - IDENTIFIES -> Workplace Risks
Company - MITIGATES -> Hazards
Company - SUPPORTS -> Employees
Company - AFFECTED_BY -> Effective Tax Rates
Company - HAS_DIFFICULTY_WITH -> Manufacture And Deliver